<a href="https://colab.research.google.com/github/premiumhomecy/premium2/blob/main/PremiumHome_Cost_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Gerekli kütüphaneleri yükle (eğer yüklü değilse)
try:
    import reportlab
    from reportlab.platypus import SimpleDocTemplate # Ekledik
except ImportError:
    print("Gerekli 'reportlab' kütüphanesi kuruluyor...")
    !pip install reportlab
    print("Kurulum tamamlandı. Lütfen hücreyi tekrar çalıştırın.")

import math
import pandas as pd
import base64
import io
import ipywidgets as widgets
from datetime import datetime
from IPython.display import display, clear_output, HTML
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import Table, TableStyle, Paragraph, Spacer, SimpleDocTemplate, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT, TA_RIGHT
from reportlab.lib.units import mm, inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import requests
from PIL import Image as PILImage

# Google Drive'ı bağla (Colab için)
from google.colab import drive
drive.mount('/content/drive')

# === THEME AND FONT SETTINGS ===
DARK_MODE = False  # User can toggle dark mode

# Register a font that supports Greek characters (e.g., DejaVuSans or FreeSans)
# IMPORTANT: Ensure FreeSans.ttf and FreeSansBold.ttf are in your Google Drive,
# for example, in a folder named 'fonts' directly under 'My Drive'.
try:
    pdfmetrics.registerFont(TTFont("FreeSans", "/content/drive/My Drive/fonts/FreeSans.ttf"))
    pdfmetrics.registerFont(TTFont("FreeSans-Bold", "/content/drive/My Drive/fonts/FreeSansBold.ttf"))
    pdfmetrics.registerFontFamily('FreeSans',
                                  normal='FreeSans',
                                  bold='FreeSans-Bold',
                                  italic='FreeSans', # Fallback to normal if no italic font is available
                                  boldItalic='FreeSans-Bold') # Fallback to bold if no bold-italic font is available
    MAIN_FONT = "FreeSans"
except Exception as e:
    print(f"UYARI: FreeSans fontları kaydedilemedi. Lütfen FreeSans.ttf ve FreeSansBold.ttf dosyalarının Google Drive'ınızda '/content/drive/My Drive/fonts/' yolunda olduğundan emin olun. Helvetica kullanılıyor. Hata: {e}")
    print(f"WARNING: Could not register FreeSans fonts. Please ensure FreeSans.ttf and FreeSansBold.ttf are in '/content/drive/My Drive/fonts/' in your Google Drive. Using Helvetica. Error: {e}")
    MAIN_FONT = "Helvetica" # Fallback font


def set_theme(dark_mode=False):
    """Sets the UI theme (light or dark)"""
    global DARK_MODE
    DARK_MODE = dark_mode

    bg_color = "#f8f9fa" if not dark_mode else "#1a202c"
    text_color = "#2d3748" if not dark_mode else "#e2e8f0"
    primary_color = "#3182ce" if not dark_mode else "#63b3ed"
    secondary_color = "#e2e8f0" if not dark_mode else "#4a5568"
    card_bg = "#ffffff" if not dark_mode else "#2d3748"
    border_color = "#e2e8f0" if not dark_mode else "#4a5568"
    # Added for checkbox/input text color in dark mode
    input_text_color = "#2d3748" if not dark_mode else "#e2e8f0"


    display(HTML(f"""
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600&display=swap');
        * {{
            font-family: 'Inter', sans-serif !important;
        }}

        .widget-label {{
            font-weight: 500;
            margin-top: 10px;
            color: {text_color} !important;
        }}

        h3 {{
            color: {primary_color};
            border-bottom: 2px solid {primary_color};
            padding-bottom: 5px;
            margin-top: 20px;
            font-weight: 600 !important;
        }}

        .dataframe {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
            font-size: 14px;
            background-color: {card_bg};
            color: {text_color};
        }}

        /* Modified: Simpler, less colorful dataframe table */
        .dataframe th, .dataframe td {{
            border: 1px solid {border_color};
            padding: 8px;
            text-align: left;
        }}

        .dataframe th {{
            background-color: {primary_color}; /* Primary color for header */
            color: white;
            font-weight: 600 !important;
        }}

        .dataframe tr:nth-child(even) {{
            background-color: {'#f0f4f7' if not dark_mode else '#2f3b4d'}; /* Slightly different shade for even rows */
        }}
        .dataframe tr:nth-child(odd) {{
            background-color: {card_bg}; /* Card background for odd rows */
        }}


        .pdf-button {{
            background-color: {primary_color};
            border: none;
            color: white;
            padding: 12px 24px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 15px;
            cursor: pointer;
            border-radius: 8px;
            transition: all 0.3s;
            font-weight: 500 !important;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }}

        .pdf-button:hover {{
            background-color: {'#2c5282' if not dark_mode else '#90cdf4'};
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }}

        .pdf-container {{
            margin: 25px 0;
            padding: 20px;
            background-color: {'#edf2f7' if not dark_mode else '#1a202c'};
            border-radius: 10px;
            text-align: center;
            border: 1px solid {border_color};
        }}

        .customer-section {{
            background-color: {'#ebf8ff' if not dark_mode else '#2b6cb0'};
            border: 1px solid {primary_color};
            border-radius: 8px;
            padding: 15px;
            margin-bottom: 20px;
        }}

        .warning {{
            color: {'#e53e3e' if not dark_mode else '#fc8181'};
            font-weight: 500;
            background-color: {'#fff5f5' if not dark_mode else '#742a2a'};
            padding: 10px;
            border-radius: 5px;
            border: 1px solid {'#fed7d7' if not dark_mode else '#9b2c2c'};
        }}

        .section-title {{
            background-color: {primary_color};
            color: white;
            padding: 10px;
            border-radius: 8px;
            font-weight: 600 !important;
            margin-top: 20px;
        }}

        .price-table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-size: 14px;
            background-color: {card_bg};
            color: {text_color};
        }}

        .price-table th, .price-table td {{
            border: 1px solid {border_color};
            padding: 10px;
            text-align: left;
        }}

        .price-table th {{
            background-color: {primary_color};
            color: white;
        }}

        .price-table tr:nth-child(even) {{
            background-color: {'#f7fafc' if not dark_mode else '#2d3748'};
        }}

        body {{
            background-color: {bg_color};
            color: {text_color};
        }}

        .widget-text input, .widget-dropdown select {{
            background-color: {card_bg};
            color: {input_text_color}; /* Ensure text is visible in input fields */
            border: 1px solid {border_color};
        }}
        /* Fixed: Ensure checkbox text color is visible */
        .widget-checkbox label {{
            color: {text_color} !important;
        }}

        .widget-checkbox input[type="checkbox"] {{
            filter: {'none' if not dark_mode else 'invert(80%)'};
        }}

        .card {{
            background-color: {card_bg};
            border-radius: 8px;
            padding: 15px;
            margin-bottom: 15px;
            border: 1px solid {border_color};
            box-shadow: 0 1px 3px rgba(0,0,0,0.1);
        }}
    </style>
    """))

# Initialize theme
set_theme(DARK_MODE)

# Dark mode toggle button
def toggle_dark_mode(b):
    set_theme(not DARK_MODE)

dark_mode_button = widgets.ToggleButton(
    value=DARK_MODE,
    description='🌙 Dark Mode' if DARK_MODE else '☀️ Light Mode',
    tooltip='Toggle dark/light mode',
    button_style='',
    icon='moon' if DARK_MODE else 'sun'
)
dark_mode_button.observe(lambda change: toggle_dark_mode(change), 'value')

# === COMPANY INFORMATION ===
# Logo URL updated
LOGO_URL = "https://drive.google.com/uc?export=download&id=1RD27Gas035iUqe4Ucl3phFwxZPWfzlzn"
LINKTREE_URL = "https://linktr.ee/premiumplushome?utm_source=linktree_admin_share"
COMPANY_INFO = {
    "name": "PREMIUM HOME LTD",
    "address": "Iasonos 1, 1082, Nicosia Cyprus",
    "email": "info@premiumpluscy.eu",
    "phone": "+35722584081, +35797550946",
    "website": "www.premiumpluscy.eu",
    "linktree": LINKTREE_URL
}

# === PRICE DEFINITIONS ===
FIYATLAR = {
    # Material Prices
    "heavy_steel_m2": 400.00,
    # "light_steel_m2": 200.00, # Removed as light steel is now calculated by profile
    "sandwich_panel_m2": 22.00,
    "plywood_piece": 44.44,
    "aluminum_window_piece": 250.00,
    "sliding_glass_door_piece": 300.00,
    "wc_window_piece": 120.00,
    "wc_sliding_door_piece": 150.00,
    "door_piece": 280.00,
    "kitchen_installation_piece": 1500.00,
    "shower_wc_installation_piece": 1000.00,
    "connection_element_m2": 1.50,
    "transportation": 500.00,
    "floor_heating_m2": 50.00,
    "wc_ceramic_m2": 20.00 + (650/45.5), # This calculation seems specific, kept as is

    # Labor Prices
    "welding_labor_m2": 160.00,
    "panel_assembly_labor_m2": 5.00,
    "plasterboard_material_m2": 20.00,
    "plasterboard_labor_m2": 80.00,
    "plywood_flooring_labor_m2": 11.11,
    "door_window_assembly_labor_piece": 50.00,

    # Installation Prices
    "electrical_installation_price": 1200.00,
    "plumbing_installation_price": 1300.00,

    # Solar Price (Fixed for now, can be updated)
    "solar_5kw_price": 9000.00,
    "solar_7_2kw_price": 12000.00, # Example, adjust as needed
    "solar_11kw_price": 18000.00, # Example, adjust as needed

    # Steel Profile Prices (per meter, for Light Steel calculation)
    "profile_100x100_price_m": 25.00,
    "profile_100x50_price_m": 20.00,
    "profile_40x60_price_m": 15.00,
    "profile_50x50_price_m": 18.00,
    "profile_30x30_price_m": 12.00,
    "profile_HEA160_price_m": 35.00,
    "standard_profile_length_m": 6.0, # Assumed standard length for each profile piece entered by count
}

FIRE_RATE = 0.05
VAT_RATE = 0.19

# === CUSTOMER INFORMATION WIDGETS ===
customer_title = widgets.HTML(value="<div class='section-title'>CUSTOMER INFORMATION (Optional)</div>")
customer_note = widgets.HTML(value="<div class='warning'>Note: Customer information is not mandatory. If left blank, it will be marked as 'GENERAL'.</div>")

customer_name = widgets.Text(
    description="Full Name:",
    value="GENERAL",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

customer_company = widgets.Text(
    description="Company:",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

customer_address = widgets.Text(
    description="Address:",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

customer_city = widgets.Text(
    description="City:",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

customer_phone = widgets.Text(
    description="Phone:",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

customer_email = widgets.Text(
    description="Email:",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

# === PROJECT INFORMATION WIDGETS ===
structure_type = widgets.ToggleButtons(
    options=['Light Steel', 'Heavy Steel'],
    description='Structure Type:',
    button_style='primary',
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

# Plasterboard Option
plasterboard_option = widgets.Checkbox(
    value=True,
    description="Include Interior Plasterboard",
    indent=False,
    style={'description_width': '120px'}
)

width_input = widgets.FloatText(
    value=10,
    description="Width (m):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

length_input = widgets.FloatText(
    value=8,
    description="Length (m):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

height_input = widgets.FloatText(
    value=2.6,
    description="Height (m):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

# Room Configuration Dropdown
room_config_input = widgets.Dropdown(
    options=[
        "1 room",
        "1 room, 1 bathroom, 1 kitchen",
        "2 rooms, 1 bathroom, 1 kitchen",
        "3 rooms, 2 bathrooms, 1 kitchen",
        "Empty Model (shell only)",
        "Custom"
    ],
    value="1 room, 1 bathroom, 1 kitchen",
    description="Room Configuration:",
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

# Steel Profile Quantity Inputs (RE-ADDED)
profile_count_label = widgets.HTML(value="<div class='section-title'>STEEL PROFILE QUANTITIES (for Light Steel)</div>")

profile_100x100_count = widgets.IntText(
    value=20,
    description="100x100 (pieces):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

profile_100x50_count = widgets.IntText(
    value=30,
    description="100x50 (pieces):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

profile_40x60_count = widgets.IntText(
    value=50,
    description="40x60 (pieces):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

profile_50x50_count = widgets.IntText(
    value=40,
    description="50x50 (pieces):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

profile_30x30_count = widgets.IntText(
    value=60,
    description="30x30 (pieces):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

profile_HEA160_count = widgets.IntText(
    value=0, # HEA usually for heavy loads, default 0 for light steel
    description="HEA160 (pieces):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

# Steel Profile Quantities Section (MOVED UP FOR DEFINITION BEFORE USE)
steel_profile_section = widgets.VBox([
    profile_count_label,
    widgets.HBox([profile_100x100_count, profile_100x50_count, profile_40x60_count]),
    widgets.HBox([profile_50x50_count, profile_30x30_count, profile_HEA160_count])
], layout=widgets.Layout(margin='0 0 20px 0'))


# Function to toggle visibility of steel profile inputs based on structure_type
def toggle_steel_profile_visibility(change):
    if change['new'] == 'Light Steel':
        steel_profile_section.layout.display = 'flex'
    else:
        steel_profile_section.layout.display = 'none'

# Observe changes in structure_type
structure_type.observe(toggle_steel_profile_visibility, names='value')

# Initial call to set visibility based on default value
toggle_steel_profile_visibility({'new': structure_type.value})


# Additional Features
heating_option = widgets.Checkbox(
    value=False,
    description="Include Floor Heating (50€/m²)",
    indent=False,
    style={'description_width': '120px'}
)

solar_option = widgets.Checkbox(
    value=False,
    description="Include Solar Energy System",
    indent=False,
    style={'description_width': '120px'}
)

solar_capacity = widgets.Dropdown(
    options=[('5 kW', 5), ('7.2 kW', 7.2), ('11 kW', 11)],
    value=5,
    description="Capacity:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

solar_price_display = widgets.FloatText(
    value=FIYATLAR["solar_5kw_price"], # Default price for 5kW
    description="Solar System Price (€):",
    disabled=True,
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

def update_solar_price_display(change):
    if change.new == 5:
        solar_price_display.value = FIYATLAR["solar_5kw_price"]
    elif change.new == 7.2:
        solar_price_display.value = FIYATLAR["solar_7_2kw_price"]
    elif change.new == 11:
        solar_price_display.value = FIYATLAR["solar_11kw_price"]
solar_capacity.observe(update_solar_price_display, names='value')


# Windows and Doors Section
window_input = widgets.IntText(
    value=4,
    description="Window Count:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

window_size = widgets.Text(
    value="100x100 cm",
    description="Size:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

sliding_door_input = widgets.IntText(
    value=0,
    description="Sliding Glass Door:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

sliding_door_size = widgets.Text(
    value="200x200 cm",
    description="Size:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

wc_window_input = widgets.IntText(
    value=1,
    description="WC Window Count:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

wc_window_size = widgets.Text(
    value="60x50 cm", # UPDATED WC WINDOW SIZE
    description="Size:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

wc_sliding_door_input = widgets.IntText(
    value=0,
    description="WC Sliding Door:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

wc_sliding_door_size = widgets.Text(
    value="140x70 cm",
    description="Size:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

door_input = widgets.IntText(
    value=2,
    description="Door Count:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

door_size = widgets.Text(
    value="90x210 cm",
    description="Size:",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

# Additional Equipment
kitchen_input = widgets.Checkbox(
    value=True,
    description="Include Kitchen",
    indent=False,
    style={'description_width': '120px'}
)

shower_input = widgets.Checkbox(
    value=True,
    description="Include Shower/WC",
    indent=False,
    style={'description_width': '120px'}
)

wc_ceramic_input = widgets.Checkbox(
    value=False,
    description="WC Ceramic Floor/Walls",
    indent=False,
    style={'description_width': '120px'}
)

wc_ceramic_area = widgets.FloatText(
    value=0,
    description="WC Ceramic Area (m²):",
    style={'description_width': '120px'},
    layout={'width': '200px'}
)

electrical_installation_input = widgets.Checkbox(
    value=False,
    description="Include Electrical Installation",
    indent=False,
    style={'description_width': '120px'}
)

plumbing_installation_input = widgets.Checkbox(
    value=False,
    description="Include Plumbing Installation",
    indent=False,
    style={'description_width': '120px'}
)

transportation_input = widgets.Checkbox(
    value=False,
    description="Include Transportation (500€)",
    indent=False,
    style={'description_width': '120px'}
)

# Financial Settings
profit_rate_input = widgets.FloatSlider(
    value=0.20,
    min=0.0,
    max=0.50,
    step=0.01,
    description="Profit Rate:",
    readout_format='.0%',
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

vat_rate_input = widgets.FloatSlider(
    value=VAT_RATE,
    min=0.0,
    max=0.25,
    step=0.01,
    description="VAT Rate:",
    readout_format='.0%',
    style={'description_width': '120px'},
    layout={'width': '400px'}
)

# Customer Notes
customer_notes = widgets.Textarea(
    value='',
    description='Customer Notes:',
    style={'description_width': '120px'},
    layout={'width': '95%', 'height': '100px'}
)

# === CALCULATION FUNCTIONS ===
def calculate_area(width, length, height):
    floor_area = width * length
    wall_area = math.ceil(2 * (width + length) * height)
    roof_area = floor_area
    return {"floor": floor_area, "wall": wall_area, "roof": roof_area}

def format_currency(value):
    """Formats currency professionally: €32,500.00"""
    if value >= 1000:
        return f"€{value:,.2f}"
    return f"€{value:.2f}"

def calculate():
    width, length, height = width_input.value, length_input.value, height_input.value
    window_count, sliding_door_count = window_input.value, sliding_door_input.value
    wc_window_count, wc_sliding_door_count = wc_window_input.value, wc_sliding_door_input.value
    door_count = door_input.value

    areas = calculate_area(width, length, height)
    floor_area = areas["floor"]
    wall_area = areas["wall"]
    roof_area = areas["roof"]

    costs = []
    profile_analysis_details = [] # Initialize for Light Steel analysis

    # Steel Structure
    if structure_type.value == 'Heavy Steel':
        total_price = floor_area * FIYATLAR["heavy_steel_m2"]
        costs.append({
            'Item': 'Heavy Steel Construction',
            'Quantity': f'{floor_area:.2f} m²',
            'Unit Price (€)': format_currency(FIYATLAR["heavy_steel_m2"]),
            'Total (€)': format_currency(total_price)
        })

        total_price = floor_area * FIYATLAR["welding_labor_m2"]
        costs.append({
            'Item': 'Steel Welding Labor',
            'Quantity': f'{floor_area:.2f} m²',
            'Unit Price (€)': format_currency(FIYATLAR["welding_labor_m2"]),
            'Total (€)': format_currency(total_price)
        })
        profile_analysis_details = pd.DataFrame() # Ensure empty for Heavy Steel
    else: # Light Steel
        # Steel Profile Calculation (RE-ADDED)
        input_profile_counts = {
            "100x100": profile_100x100_count.value,
            "100x50": profile_100x50_count.value,
            "40x60": profile_40x60_count.value,
            "50x50": profile_50x50_count.value,
            "30x30": profile_30x30_count.value,
            "HEA160": profile_HEA160_count.value,
        }

        total_light_steel_cost = 0
        for profile_type, count in input_profile_counts.items():
            if count > 0:
                price_per_meter_key = f"profile_{profile_type}_price_m"
                if price_per_meter_key in FIYATLAR:
                    price_per_meter = FIYATLAR[price_per_meter_key]
                    total_length_m = count * FIYATLAR["standard_profile_length_m"]
                    total_price_for_profile = total_length_m * price_per_meter
                    total_light_steel_cost += total_price_for_profile

                    profile_analysis_details.append({
                        'Profile Type': profile_type,
                        'Count (pieces)': count,
                        'Length per piece (m)': FIYATLAR["standard_profile_length_m"],
                        'Total Length (m)': f'{total_length_m:.2f}',
                        'Unit Price (€/m)': format_currency(price_per_meter),
                        'Total (€)': format_currency(total_price_for_profile)
                    })

        costs.append({
            'Item': 'Light Steel Construction (Profiles)',
            'Quantity': f'{floor_area:.2f} m² (Estimated)', # Quantity can be estimated m2 or total length
            'Unit Price (€)': 'N/A', # Unit price is complex for summed profiles
            'Total (€)': format_currency(total_light_steel_cost)
        })

        # Welding labor is still applicable for light steel
        total_price = floor_area * FIYATLAR["welding_labor_m2"]
        costs.append({
            'Item': 'Steel Welding Labor',
            'Quantity': f'{floor_area:.2f} m²',
            'Unit Price (€)': format_currency(FIYATLAR["welding_labor_m2"]),
            'Total (€)': format_currency(total_price)
        })
        profile_analysis_details = pd.DataFrame(profile_analysis_details) # Convert to DataFrame here

    # Cladding and Insulation
    total_price = roof_area * FIYATLAR["sandwich_panel_m2"]
    costs.append({
        'Item': 'Roof (Sandwich Panel)',
        'Quantity': f'{roof_area:.2f} m²',
        'Unit Price (€)': format_currency(FIYATLAR["sandwich_panel_m2"]),
        'Total (€)': format_currency(total_price)
    })

    total_price = wall_area * FIYATLAR["sandwich_panel_m2"]
    costs.append({
        'Item': 'Facade (Sandwich Panel)',
        'Quantity': f'{wall_area:.2f} m²',
        'Unit Price (€)': format_currency(FIYATLAR["sandwich_panel_m2"]),
        'Total (€)': format_currency(total_price)
    })

    total_price = (wall_area + roof_area) * FIYATLAR["panel_assembly_labor_m2"]
    costs.append({
        'Item': "Panel Assembly Labor",
        'Quantity': f"{(wall_area + roof_area):.2f} m²",
        'Unit Price (€)': format_currency(FIYATLAR["panel_assembly_labor_m2"]),
        'Total (€)': format_currency(total_price)
    })

    # Interior and Flooring
    if plasterboard_option.value:
        plasterboard_area = wall_area + roof_area
        total_price = plasterboard_area * FIYATLAR["plasterboard_material_m2"]
        costs.append({
            'Item': 'Plasterboard Material',
            'Quantity': f'{plasterboard_area:.2f} m²',
            'Unit Price (€)': format_currency(FIYATLAR["plasterboard_material_m2"]),
            'Total (€)': format_currency(total_price)
        })

        total_price = plasterboard_area * FIYATLAR["plasterboard_labor_m2"]
        costs.append({
            'Item': 'Plasterboard Labor',
            'Quantity': f'{plasterboard_area:.2f} m²',
            'Unit Price (€)': format_currency(FIYATLAR["plasterboard_labor_m2"]),
            'Total (€)': format_currency(total_price)
        })

    plywood_pieces_needed = math.ceil(floor_area / (1.22 * 2.44))
    total_price = plywood_pieces_needed * FIYATLAR["plywood_piece"]
    costs.append({
        'Item': 'Floor (Plywood Material)',
        'Quantity': plywood_pieces_needed,
        'Unit Price (€)': format_currency(FIYATLAR["plywood_piece"]),
        'Total (€)': format_currency(total_price)
    })

    total_price = floor_area * FIYATLAR["plywood_flooring_labor_m2"]
    costs.append({
        'Item': 'Floor (Plywood Labor)',
        'Quantity': f'{floor_area:.2f} m²',
        'Unit Price (€)': format_currency(FIYATLAR["plywood_flooring_labor_m2"]),
        'Total (€)': format_currency(total_price)
    })

    # Floor Heating
    if heating_option.value:
        total_price = floor_area * FIYATLAR["floor_heating_m2"]
        costs.append({
            'Item': 'Floor Heating System',
            'Quantity': f'{floor_area:.2f} m²',
            'Unit Price (€)': format_currency(FIYATLAR["floor_heating_m2"]),
            'Total (€)': format_currency(total_price)
        })

    # Solar Energy System (handled separately for total and proposal)
    solar_system_cost = 0
    if solar_option.value:
        if solar_capacity.value == 5:
            solar_system_cost = FIYATLAR['solar_5kw_price']
        elif solar_capacity.value == 7.2:
            solar_system_cost = FIYATLAR['solar_7_2kw_price']
        elif solar_capacity.value == 11:
            solar_system_cost = FIYATLAR['solar_11kw_price']
        # The solar system cost is NOT added to 'costs' list for the main subtotal calculation
        # It will be displayed separately in the financial summary and proposal.


    # Windows and Doors
    if window_count > 0:
        total_price = window_count * FIYATLAR["aluminum_window_piece"]
        costs.append({
            'Item': f'Window ({window_size.value})',
            'Quantity': window_count,
            'Unit Price (€)': format_currency(FIYATLAR["aluminum_window_piece"]),
            'Total (€)': format_currency(total_price)
        })

    if sliding_door_count > 0:
        total_price = sliding_door_count * FIYATLAR["sliding_glass_door_piece"]
        costs.append({
            'Item': f'Sliding Glass Door ({sliding_door_size.value})',
            'Quantity': sliding_door_count,
            'Unit Price (€)': format_currency(FIYATLAR["sliding_glass_door_piece"]),
            'Total (€)': format_currency(total_price)
        })

    if wc_window_count > 0:
        total_price = wc_window_count * FIYATLAR["wc_window_piece"]
        costs.append({
            'Item': f'WC Window ({wc_window_size.value})',
            'Quantity': wc_window_count,
            'Unit Price (€)': format_currency(FIYATLAR["wc_window_piece"]),
            'Total (€)': format_currency(total_price)
        })

    if wc_sliding_door_count > 0:
        total_price = wc_sliding_door_count * FIYATLAR["wc_sliding_door_piece"]
        costs.append({
            'Item': f'WC Sliding Door ({wc_sliding_door_size.value})',
            'Quantity': wc_sliding_door_count,
            'Unit Price (€)': format_currency(FIYATLAR["wc_sliding_door_piece"]),
            'Total (€)': format_currency(total_price)
        })

    if door_count > 0:
        total_price = door_count * FIYATLAR["door_piece"]
        costs.append({
            'Item': f'Door ({door_size.value})',
            'Quantity': door_count,
            'Unit Price (€)': format_currency(FIYATLAR["door_piece"]),
            'Total (€)': format_currency(total_price)
        })

    total_door_window_count = window_count + sliding_door_count + wc_window_count + wc_sliding_door_count + door_count
    if total_door_window_count > 0:
        total_price = total_door_window_count * FIYATLAR["door_window_assembly_labor_piece"]
        costs.append({
            'Item': 'Door/Window Assembly Labor',
            'Quantity': total_door_window_count,
            'Unit Price (€)': format_currency(FIYATLAR["door_window_assembly_labor_piece"]),
            'Total (€)': format_currency(total_price)
        })

    # Other Items
    total_price = floor_area * FIYATLAR["connection_element_m2"]
    costs.append({
        'Item': "Connection Elements",
        'Quantity': f"{floor_area:.2f} m²",
        'Unit Price (€)': format_currency(FIYATLAR["connection_element_m2"]),
        'Total (€)': format_currency(total_price)
    })

    if kitchen_input.value:
        costs.append({
            'Item': 'Kitchen Installation',
            'Quantity': 1,
            'Unit Price (€)': format_currency(FIYATLAR["kitchen_installation_piece"]),
            'Total (€)': format_currency(FIYATLAR["kitchen_installation_piece"])
        })

    if shower_input.value:
        costs.append({
            'Item': 'Shower/WC Installation',
            'Quantity': 1,
            'Unit Price (€)': format_currency(FIYATLAR["shower_wc_installation_piece"]),
            'Total (€)': format_currency(FIYATLAR["shower_wc_installation_piece"])
        })

    if wc_ceramic_input.value and wc_ceramic_area.value > 0:
        total_price = wc_ceramic_area.value * FIYATLAR["wc_ceramic_m2"]
        costs.append({
            'Item': 'WC Ceramic Floor/Walls',
            'Quantity': f"{wc_ceramic_area.value:.2f} m²",
            'Unit Price (€)': format_currency(FIYATLAR["wc_ceramic_m2"]),
            'Total (€)': format_currency(total_price)
        })

    if electrical_installation_input.value:
        costs.append({
            'Item': 'Electrical Installation',
            'Quantity': 1,
            'Unit Price (€)': format_currency(FIYATLAR["electrical_installation_price"]),
            'Total (€)': format_currency(FIYATLAR["electrical_installation_price"])
        })

    if plumbing_installation_input.value:
        costs.append({
            'Item': 'Plumbing Installation',
            'Quantity': 1,
            'Unit Price (€)': format_currency(FIYATLAR["plumbing_installation_price"]),
            'Total (€)': format_currency(FIYATLAR["plumbing_installation_price"])
        })

    if transportation_input.value:
        costs.append({
            'Item': 'Transportation',
            'Quantity': 1,
            'Unit Price (€)': format_currency(FIYATLAR["transportation"]),
            'Total (€)': format_currency(FIYATLAR["transportation"])
        })

    # Financial Calculations for the main proposal (excluding solar)
    subtotal = sum([float(item['Total (€)'].replace('€', '').replace(',', '')) for item in costs if 'Total (€)' in item])

    waste_cost = subtotal * FIRE_RATE
    total_cost_excluding_solar = subtotal + waste_cost
    profit = total_cost_excluding_solar * profit_rate_input.value
    vat_base_excluding_solar = total_cost_excluding_solar + profit
    vat_excluding_solar = vat_base_excluding_solar * vat_rate_input.value
    sales_price_excluding_solar = vat_base_excluding_solar + vat_excluding_solar

    # Total Sales Price including solar (for overall display, but main proposal price excludes it)
    total_sales_price_with_solar = sales_price_excluding_solar + solar_system_cost


    financial_summary_data = [
        ["Subtotal", subtotal],
        [f"Waste Cost (%{FIRE_RATE*100:.0f})", waste_cost],
        ["Total Cost (Including Waste)", total_cost_excluding_solar],
        [f"Profit (%{profit_rate_input.value*100:.0f})", profit],
        [f"VAT (%{vat_rate_input.value*100:.0f})", vat_excluding_solar],
        ["Sales Price (VAT Included) - Excluding Solar", sales_price_excluding_solar]
    ]

    # Add Solar System as a separate line in financial summary if included
    if solar_option.value:
        financial_summary_data.append([
            f"Solar Energy System ({solar_capacity.value} kW)", solar_system_cost
        ])
        financial_summary_data.append([
            "GRAND TOTAL (Incl. Solar & VAT)", total_sales_price_with_solar
        ])


    # Formatted financial summary
    formatted_financial_summary = []
    for item, amount in financial_summary_data:
        formatted_financial_summary.append({
            'Item': item,
            'Amount (€)': format_currency(amount)
        })

    # Process customer information
    customer_name_value = customer_name.value.strip() or "GENERAL"

    return {
        'cost_breakdown': pd.DataFrame(costs),
        'financial_summary': pd.DataFrame(formatted_financial_summary),
        'profile_analysis': profile_analysis_details,
        'notes': customer_notes.value,
        'sales_price_excluding_solar': sales_price_excluding_solar, # Main proposal price
        'total_sales_price_with_solar': total_sales_price_with_solar, # Grand total for internal report
        'area': floor_area,
        'width': width,
        'length': length,
        'height': height,
        'customer_info': {
            'name': customer_name_value,
            'company': customer_company.value or "",
            'address': customer_address.value or "",
            'city': customer_city.value or "",
            'phone': customer_phone.value or "",
            'email': customer_email.value or ""
        },
        'project_details': {
            'width': width,
            'length': length,
            'height': height,
            'area': floor_area,
            'structure_type': structure_type.value,
            'plasterboard': plasterboard_option.value,
            'window_count': window_input.value,
            'window_size': window_size.value,
            'sliding_door_count': sliding_door_input.value,
            'sliding_door_size': sliding_door_size.value,
            'wc_window_count': wc_window_input.value,
            'wc_window_size': wc_window_size.value,
            'wc_sliding_door_count': wc_sliding_door_input.value,
            'wc_sliding_door_size': wc_sliding_door_size.value,
            'door_count': door_input.value,
            'door_size': door_size.value,
            'kitchen': kitchen_input.value,
            'shower': shower_input.value,
            'wc_ceramic': wc_ceramic_input.value,
            'wc_ceramic_area': wc_ceramic_area.value,
            'electrical': electrical_installation_input.value,
            'plumbing': plumbing_installation_input.value,
            'transportation': transportation_input.value,
            'heating': heating_option.value,
            'solar': solar_option.value,
            'solar_kw': solar_capacity.value,
            'solar_price': solar_system_cost, # The actual fixed price
            'vat_rate': vat_rate_input.value,
            'profit_rate': profit_rate_input.value,
            'room_configuration': room_config_input.value
        }
    }

# === PDF CREATION FUNCTIONS ===
def create_pdf_download_link(pdf_bytes, filename):
    """Creates a download link for PDF files"""
    b64 = base64.b64encode(pdf_bytes).decode()
    return f'<a class="pdf-button" href="data:application/pdf;base64,{b64}" download="{filename}">Download {filename}</a>'

def get_company_logo(width_mm=40): # width in mm for better control
    """Gets the company logo and returns it in base64 format"""
    try:
        response = requests.get(LOGO_URL)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        img = PILImage.open(io.BytesIO(response.content))

        # Calculate height based on aspect ratio
        aspect_ratio = img.width / img.height
        width_pixels = int(width_mm * (96 / 25.4)) # Convert mm to pixels (assuming 96 dpi)
        height_pixels = int(width_pixels / aspect_ratio)

        img = img.resize((width_pixels, height_pixels), PILImage.LANCZOS)
        buffered = io.BytesIO()
        img.save(buffered, format="PNG") # Use PNG for transparency if applicable
        return base64.b64encode(buffered.getvalue()).decode()
    except requests.exceptions.RequestException as e:
        print(f"Error loading logo from URL: {e}")
        return None
    except Exception as e:
        print(f"Error processing logo: {e}")
        return None


def draw_header(canvas, doc, logo_data):
    """Draws the page header (logo and company information)"""
    # Logo
    if logo_data:
        logo = Image(io.BytesIO(base64.b64decode(logo_data)))
        logo_width_mm = 40
        logo_height_mm = logo_width_mm / (logo.drawWidth / logo.drawHeight) if logo.drawHeight else 0 # Maintain aspect ratio
        logo.drawWidth = logo_width_mm * mm
        logo.drawHeight = logo_height_mm * mm

        # Position at top right, with some margin
        x_position = A4[0] - doc.rightMargin - logo.drawWidth
        y_position = A4[1] - doc.topMargin + 5*mm # 5mm above top margin to align with page top
        logo.drawOn(canvas, x_position, y_position)

def draw_footer(canvas, doc, lang="en", report_type="customer"): # Added report_type parameter
    """Draws the page footer with company info and Linktree catalog link (conditional)"""
    # Company contact information
    footer_text = f"{COMPANY_INFO['address']} | {COMPANY_INFO['email']} | {COMPANY_INFO['phone']} | {COMPANY_INFO['website']}"
    canvas.setFont(f"{MAIN_FONT}-Bold", 9)
    canvas.drawCentredString(doc.width/2 + doc.leftMargin, 20*mm, footer_text)

    # Linktree catalog link - ONLY FOR CUSTOMER PROPOSAL
    if report_type == "customer":
        if lang == "en":
            catalog_text = f"Catalog: {COMPANY_INFO['linktree']}"
        elif lang == "tr":
            catalog_text = f"Katalog: {COMPANY_INFO['linktree']}"
        elif lang == "gr": # Assuming the same linktree for Greek
            catalog_text = f"Κατάλογος: {COMPANY_INFO['linktree']}"
        canvas.setFont(MAIN_FONT, 8)
        canvas.drawCentredString(doc.width/2 + doc.leftMargin, 15*mm, catalog_text)

    # Page number
    page_num = canvas.getPageNumber()
    if lang == "en":
        page_str = f"Page {page_num}"
    elif lang == "tr":
        page_str = f"Sayfa {page_num}"
    elif lang == "gr":
        page_str = f"Σελίδα {page_num}"
    canvas.setFont(MAIN_FONT, 9)
    canvas.drawRightString(doc.width + doc.leftMargin, 10*mm, page_str)

def create_customer_proposal_pdf(sales_price_excluding_solar, project_details, notes, customer_info, logo_data):
    """Creates a proposal PDF for the customer (English and Greek)"""
    buffer = io.BytesIO()
    doc = SimpleDocTemplate(
        buffer,
        pagesize=A4,
        rightMargin=15*mm,
        leftMargin=15*mm,
        topMargin=40*mm, # Increased top margin for header
        bottomMargin=25*mm # Increased bottom margin for footer
    )

    def _header_footer(canvas, doc):
        draw_header(canvas, doc, logo_data)
        draw_footer(canvas, doc, lang="gr", report_type="customer") # Pass report_type

    doc.onFirstPage = _header_footer
    doc.onLaterPages = _header_footer

    styles = getSampleStyleSheet()

    # Custom styles with Greek font support (Max 14pt, Min 10pt adjustment)
    normal_style = ParagraphStyle(
        name='Normal',
        parent=styles['Normal'],
        fontSize=10, # Adjusted to 10pt
        leading=14,
        spaceAfter=4,
        fontName=MAIN_FONT
    )
    bold_style = ParagraphStyle(
        name='Bold',
        parent=styles['Normal'],
        fontSize=10, # Adjusted to 10pt
        leading=14,
        spaceAfter=4,
        fontName=f"{MAIN_FONT}-Bold"
    )
    title_style = ParagraphStyle(
        name='Title',
        parent=styles['Heading1'],
        fontSize=14, # Adjusted to 14pt (Max)
        alignment=TA_CENTER,
        spaceAfter=14,
        fontName=f"{MAIN_FONT}-Bold",
        textColor=colors.HexColor("#3182ce")
    )
    section_heading_style = ParagraphStyle(
        name='SectionHeading',
        parent=styles['Heading2'],
        fontSize=12, # Adjusted to 12pt
        spaceBefore=12,
        spaceAfter=6,
        fontName=f"{MAIN_FONT}-Bold",
        textColor=colors.HexColor("#3182ce")
    )
    price_style = ParagraphStyle(
        name='Price',
        parent=styles['Heading1'],
        fontSize=18, # Adjusted to 18pt (slightly larger than 14pt title, but acceptable for price highlight)
        alignment=TA_CENTER,
        spaceAfter=12,
        fontName=f"{MAIN_FONT}-Bold",
        textColor=colors.HexColor("#e53e3e")
    )
    small_note_style = ParagraphStyle(
        name='SmallNote',
        parent=styles['Normal'],
        fontSize=8, # Smaller font for notes
        leading=10,
        spaceAfter=2,
        fontName=MAIN_FONT,
        textColor=colors.grey
    )
    list_item_style = ParagraphStyle(
        name='ListItem',
        parent=styles['Normal'],
        fontSize=10,
        leading=14,
        spaceAfter=2,
        leftIndent=12,
        fontName=MAIN_FONT
    )

    # NEW: Style for the red solar note
    solar_note_style = ParagraphStyle(
        name='SolarNoteStyle',
        parent=small_note_style, # Inherit size etc.
        textColor=colors.red,
        fontName=MAIN_FONT # Base font, bolding will come from <b>
    )

    elements = []

    # Title
    elements.append(Paragraph("PREFABRICATED HOUSE PROPOSAL / ΠΡΟΤΑΣΗ ΠΡΟΚΑΤΑΣΚΕΥΑΣΜΕΝΟΥ ΣΠΙΤΙΟΥ", title_style))
    elements.append(Paragraph(f"Date / Ημερομηνία: {datetime.now().strftime('%d/%m/%Y')}", normal_style))
    elements.append(Spacer(1, 12))

    # Customer Information
    elements.append(Paragraph("CUSTOMER INFORMATION / ΠΛΗΡΟΦΟΡΙΕΣ ΠΕΛΑΤΗ", section_heading_style))
    customer_data = [
        ["Name / Όνομα:", customer_info['name']],
        ["Company / Εταιρεία:", customer_info['company']],
        ["Address / Διεύθυνση:", customer_info['address']],
        ["City / Πόλη:", customer_info['city']],
        ["Phone / Τηλέφωνο:", customer_info['phone']],
        ["Email / Ηλεκτρονικό Ταχυδρομείο:", customer_info['email']]
    ]
    # Filter out empty lines for cleaner look
    customer_data_filtered = [[Paragraph(f"{label} <font name='{MAIN_FONT}-Bold'>{value}</font>", normal_style)] for label, value in customer_data if value.strip()]

    # Using a table for customer info for better alignment
    customer_table = Table(customer_data_filtered, colWidths=[doc.width])
    customer_table.setStyle(TableStyle([
        ('ALIGN', (0,0), (-1,-1), 'LEFT'),
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(customer_table)
    elements.append(Spacer(1, 12))

    # Project Details
    elements.append(Paragraph("PROJECT DETAILS / ΣΤΟΙΧΕΙΑ ΕΡΓΟΥ", section_heading_style))
    project_data = [
        ["Dimensions / Διαστάσεις:", f"{project_details['width']}m x {project_details['length']}m x {project_details['height']}m"],
        ["Total Area / Συνολική Έκταση:", f"{project_details['area']:.2f} m²"],
        ["Room Configuration / Διαμόρφωση Δωματίων:", project_details['room_configuration']],
        ["Estimated Delivery Time / Εκτιμώμενος Χρόνος Παράδοσης:", f"{int(project_details['area'] / 5) + 10} days / ημέρες"]
    ]
    project_table_data = [[Paragraph(label, normal_style), Paragraph(f"<font name='{MAIN_FONT}-Bold'>{value}</font>", normal_style)] for label, value in project_data]
    project_table = Table(project_table_data, colWidths=[doc.width/2, doc.width/2])
    project_table.setStyle(TableStyle([
        ('ALIGN', (0,0), (-1,-1), 'LEFT'),
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(project_table)
    elements.append(Spacer(1, 12))

    # Technical Specifications
    elements.append(Paragraph("TECHNICAL SPECIFICATIONS / ΤΕΧΝΙΚΑ ΧΑΡΑΚΤΗΡΙΣΤΙΚΑ", section_heading_style))
    specs_data = [
        ["Structure Type / Τύπος Κατασκευής:", project_details['structure_type']],
        ["Interior Plasterboard / Εσωτερικό Γυψοσανίδα:", 'Included / Συμπεριλαμβάνεται' if project_details['plasterboard'] else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["Windows / Παράθυρα:", f"{project_details['window_count']} units / τεμάχια ({project_details['window_size']})"],
        ["Sliding Glass Doors / Συρόμενες Γυάλινες Πόρτες:", f"{project_details['sliding_door_count']} units / τεμάχια ({project_details['sliding_door_size']})"],
        ["WC Windows / Παράθυρα WC:", f"{project_details['wc_window_count']} units / τεμάχια ({project_details['wc_window_size']})"],
        ["WC Sliding Doors / Συρόμενες Πόρτες WC:", f"{project_details['wc_sliding_door_count']} units / τεμάχια ({project_details['wc_sliding_door_size']})"],
        ["Doors / Πόρτες:", f"{project_details['door_count']} units / τεμάχια ({project_details['door_size']})"],
        ["Kitchen Installation / Εγκατάσταση Κουζίνας:", 'Included / Συμπεριλαμβάνεται' if project_details['kitchen'] else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["Shower/WC Installation / Εγκατάσταση Ντους/WC:", 'Included / Συμπεριλαμβάνεται' if project_details['shower'] else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["WC Ceramic Floor/Walls / Κεραμικά Δαπέδου/Τοίχων WC:", 'Included / Συμπεριλαμβάνεται' if project_details['wc_ceramic'] and project_details['wc_ceramic_area'] > 0 else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["Electrical Installation / Ηλεκτρολογική Εγκατάσταση:", 'Included / Συμπεριλαμβάνεται' if project_details['electrical'] else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["Plumbing Installation / Υδραυλική Eγκατάσταση:", 'Included / Συμπεριλαμβάνεται' if project_details['plumbing'] else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["Transportation / Μεταφορά:", 'Included / Συμπεριλαμβάνεται' if project_details['transportation'] else 'Not included / Δεν συμπεριλαμβάνεται'],
        ["Floor Heating / Ενδοδαπέδια Θέρμανση:", 'Included / Συμπεριλαμβάνεται' if project_details['heating'] else 'Not included / Δεν συμπεριλαμβάνεται'],
    ]
    specs_table_data = [[Paragraph(label, normal_style), Paragraph(f"<font name='{MAIN_FONT}-Bold'>{value}</font>", normal_style)] for label, value in specs_data]
    specs_table = Table(specs_table_data, colWidths=[doc.width/1.5, doc.width/3]) # Adjust col widths for better readability
    specs_table.setStyle(TableStyle([
        ('ALIGN', (0,0), (-1,-1), 'LEFT'),
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(specs_table)
    elements.append(Spacer(1, 12))

    # Notes
    if notes.strip():
        elements.append(Paragraph("CUSTOMER SPECIAL REQUESTS AND NOTES / ΕΙΔΙΚΑ ΑΙΤΗΜΑΤΑ ΚΑΙ ΣΗΜΕΙΩΣΕΙΣ ΠΕΛΑΤΗ", section_heading_style))
        elements.append(Paragraph(notes, normal_style))
        elements.append(Spacer(1, 12))

    # Total Price for the HOUSE (excluding solar) - ALWAYS SHOWN FIRST
    elements.append(Paragraph("TOTAL SALES PRICE (VAT Included) - EXCLUDING SOLAR SYSTEM / ΣΥΝΟΛΙΚΗ ΤΙΜΗ ΠΩΛΗΣΗΣ (Περιλαμβάνει ΦΠΑ) - ΕΚΤΟΣ ΗΛΙΑΚΟΥ ΣΥΣΤΗΜΑΤΟΣ", section_heading_style))
    elements.append(Paragraph(format_currency(sales_price_excluding_solar), price_style))
    elements.append(Spacer(1, 12))

    # Solar Energy System Section - CONDITIONAL AND ON NEW PAGE
    if project_details['solar']:
        elements.append(PageBreak()) # Force new page for solar details

        elements.append(Paragraph("OPTIONAL: SOLAR ENERGY SYSTEM / ΠΡΟΑΙΡΕΤΙΚΟ: ΗΛΙΑΚΟ ΣΥΣΤΗΜΑ ΕΝΕΡΓΕΙΑΣ", title_style)) # Use title_style for main heading
        elements.append(Spacer(1, 12)) # Add some space

        elements.append(Paragraph("SYSTEM DETAILS / ΣΤΟΙΧΕΙΑ ΣΥΣΤΗΜΑΤΟΣ", section_heading_style))
        elements.append(Paragraph(f"Capacity / Ισχύς: <font name='{MAIN_FONT}-Bold'>{project_details['solar_kw']} kW</font>", normal_style))
        elements.append(Paragraph(f"Price / Τιμή: <font name='{MAIN_FONT}-Bold'>{format_currency(project_details['solar_price'])}</font>", price_style))
        elements.append(Spacer(1, 6))

        elements.append(Paragraph("Materials Included / Περιλαμβανόμενα Υλικά:", bold_style))
        # Translated to English
        elements.append(Paragraph(f"• Solar Panels ({project_details['solar_kw']} kW total capacity) / Φωτοβολταϊκά Πάνελ ({project_details['solar_kw']} kW συνολικής ισχύος)", list_item_style))
        elements.append(Paragraph("• Li-ion Batteries / Μπαταρίες ιόντων λιθίου", list_item_style))
        elements.append(Paragraph(f"• Inverter ({project_details['solar_kw']} kW) / Μετατροπέας ({project_details['solar_kw']} kW)", list_item_style))
        elements.append(Paragraph("• Cabling (25m) / Καλωδίωση (25m)", list_item_style))
        elements.append(Paragraph("• Installation / Εγκατάσταση", list_item_style))
        elements.append(Paragraph("• Steel Mounting Profiles / Μεταλλικά Προφίλ Στήριξης", list_item_style))
        elements.append(Spacer(1, 6))
        elements.append(Paragraph(
            "<b>Note:</b> This solar energy system is an independent system. Warranty conditions and warranty period are provided by other companies. / "
            "<b>Σημείωση:</b> Αυτό το ηλιακό σύστημα ενέργειας είναι ένα ανεξάρτητο σύστημα. Οι όροι εγγύησης και η περίοδος εγγύησης παρέχονται από άλλες εταιρείες.",
            solar_note_style
        ))
        elements.append(Spacer(1, 12))

        # GRAND TOTAL if solar is included
        elements.append(Paragraph("GRAND TOTAL (Including Solar & VAT) / ΣΥΝΟΛΙΚΟ ΠΟΣΟ (Περιλαμβάνει Ηλιακό & ΦΠΑ)", section_heading_style))
        elements.append(Paragraph(format_currency(project_details['total_sales_price_with_solar']), price_style)) # Use the correct key
        elements.append(Spacer(1, 12))


    # Build PDF
    doc.build(elements)

    buffer.seek(0)
    return buffer.getvalue()


def create_internal_cost_report_pdf(cost_breakdown_df, financial_summary_df, profile_analysis_df, project_details, logo_data): # profile_analysis_df re-added
    """Creates an internal cost report PDF (Turkish)."""
    buffer = io.BytesIO()
    doc = SimpleDocTemplate(
        buffer,
        pagesize=A4,
        rightMargin=15*mm,
        leftMargin=15*mm,
        topMargin=40*mm,
        bottomMargin=25*mm
    )

    def _header_footer(canvas, doc):
        draw_header(canvas, doc, logo_data)
        draw_footer(canvas, doc, lang="tr", report_type="internal") # Pass report_type

    doc.onFirstPage = _header_footer
    doc.onLaterPages = _header_footer

    styles = getSampleStyleSheet()

    # Custom styles (Max 14pt, Min 10pt adjustment)
    header_style = ParagraphStyle(
        name='Header',
        parent=styles['Normal'],
        fontSize=14, # Adjusted to 14pt
        alignment=TA_CENTER,
        spaceAfter=12,
        fontName=f"{MAIN_FONT}-Bold"
    )
    section_heading_style = ParagraphStyle(
        name='SectionHeading',
        parent=styles['Heading2'],
        fontSize=12, # Adjusted to 12pt
        spaceBefore=12,
        spaceAfter=6,
        fontName=f"{MAIN_FONT}-Bold",
        textColor=colors.HexColor("#3182ce")
    )
    normal_style = ParagraphStyle(
        name='Normal',
        parent=styles['Normal'],
        fontSize=10, # Adjusted to 10pt
        leading=14,
        spaceAfter=6,
        fontName=MAIN_FONT
    )
    table_header_style = ParagraphStyle(
        name='TableHeader',
        parent=styles['Normal'],
        fontSize=9, # Adjusted to 9pt for table headers
        fontName=f"{MAIN_FONT}-Bold",
        textColor=colors.white,
        alignment=TA_CENTER
    )
    table_cell_style = ParagraphStyle(
        name='TableCell',
        parent=styles['Normal'],
        fontSize=9, # Adjusted to 9pt for table cells
        fontName=MAIN_FONT,
        alignment=TA_LEFT
    )

    elements = []

    elements.append(Paragraph("DAHİLİ MALİYET RAPORU", header_style))
    elements.append(Paragraph(f"Tarih: {datetime.now().strftime('%d/%m/%Y')}", normal_style))
    elements.append(Spacer(1, 12))

    # Project Information
    elements.append(Paragraph("PROJE BİLGİLERİ", section_heading_style))
    elements.append(Paragraph(f"Boyutlar: {project_details['width']}m x {project_details['length']}m x {project_details['height']}m", normal_style))
    elements.append(Paragraph(f"Toplam Alan: {project_details['area']:.2f} m²", normal_style))
    elements.append(Paragraph(f"Yapı Tipi: {project_details['structure_type']}", normal_style))
    elements.append(Spacer(1, 12))

    # Cost Breakdown Table
    elements.append(Paragraph("MALİYET DAĞILIMI", section_heading_style))

    cost_data = [[
        Paragraph("Madde", table_header_style),
        Paragraph("Miktar", table_header_style),
        Paragraph("Birim Fiyat (€)", table_header_style),
        Paragraph("Toplam (€)", table_header_style)
    ]]
    for _, row in cost_breakdown_df.iterrows():
        cost_data.append([
            Paragraph(str(row['Item']), table_cell_style),
            Paragraph(str(row['Quantity']), table_cell_style),
            Paragraph(str(row['Unit Price (€)']), table_cell_style),
            Paragraph(str(row['Total (€)']), table_cell_style)
        ])

    cost_table = Table(cost_data, colWidths=[65*mm, 40*mm, 40*mm, 40*mm])
    cost_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#3182ce")), # Header background
        ('TEXTCOLOR', (0,0), (-1,0), colors.white),
        ('ALIGN', (0,0), (-1,1), 'LEFT'), # Align header cells to left
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('FONTNAME', (0,0), (-1,0), f"{MAIN_FONT}-Bold"),
        ('FONTSIZE', (0,0), (-1,0), 9),
        ('BOTTOMPADDING', (0,0), (-1,0), 6),
        ('BACKGROUND', (0,1), (-1,-1), colors.white), # All rows white background
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('BOX', (0,0), (-1,-1), 1, colors.black),
        ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor("#f7fafc")]), # Simpler row backgrounds
    ]))
    elements.append(cost_table)
    elements.append(Spacer(1, 12))

    # Steel Profile Analysis Table (RE-ADDED)
    if project_details['structure_type'] == 'Light Steel' and not profile_analysis_df.empty:
        elements.append(Paragraph("ÇELİK PROFİL ANALİZİ (Hafif Çelik)", section_heading_style))
        profile_data = [[
            Paragraph("Profil Tipi", table_header_style),
            Paragraph("Adet", table_header_style),
            Paragraph("Parça Boyu (m)", table_header_style),
            Paragraph("Toplam Boy (m)", table_header_style),
            Paragraph("Birim Fiyat (€/m)", table_header_style),
            Paragraph("Toplam (€)", table_header_style)
        ]]
        for _, row in profile_analysis_df.iterrows():
            profile_data.append([
                Paragraph(str(row['Profile Type']), table_cell_style),
                Paragraph(str(row['Count (pieces)']), table_cell_style),
                Paragraph(str(row['Length per piece (m)']), table_cell_style),
                Paragraph(str(row['Total Length (m)']), table_cell_style),
                Paragraph(str(row['Unit Price (€/m)']), table_cell_style),
                Paragraph(str(row['Total (€)']), table_cell_style)
            ])

        profile_table = Table(profile_data, colWidths=[30*mm, 20*mm, 20*mm, 25*mm, 35*mm, 35*mm])
        profile_table.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#3182ce")),
            ('TEXTCOLOR', (0,0), (-1,0), colors.white),
            ('ALIGN', (0,0), (-1,1), 'LEFT'),
            ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
            ('FONTNAME', (0,0), (-1,0), f"{MAIN_FONT}-Bold"),
            ('FONTSIZE', (0,0), (-1,0), 9),
            ('BOTTOMPADDING', (0,0), (-1,0), 6),
            ('BACKGROUND', (0,1), (-1,-1), colors.white),
            ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
            ('BOX', (0,0), (-1,-1), 1, colors.black),
            ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor("#f7fafc")]),
        ]))
        elements.append(profile_table)
        elements.append(Spacer(1, 12))


    # Financial Summary Table - ON A NEW PAGE WITH DETAILS
    elements.append(PageBreak()) # New page for financial summary
    elements.append(Paragraph("FİNANSAL ÖZET", section_heading_style))
    elements.append(Paragraph("Bu bölüm, projenin maliyet dökümünün ve nihai satış fiyatının hesaplanmasına ilişkin ayrıntılı bir özeti sunar.", normal_style))
    elements.append(Spacer(1, 6))

    financial_data = []
    for _, row in financial_summary_df.iterrows():
        financial_data.append([
            Paragraph(str(row['Item']), table_cell_style),
            Paragraph(str(row['Amount (€)']), table_cell_style)
        ])

    financial_table = Table(financial_data, colWidths=[90*mm, 85*mm])
    financial_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#3182ce")),
        ('TEXTCOLOR', (0,0), (-1,0), colors.white),
        ('ALIGN', (0,0), (-1,1), 'LEFT'), # Align header cells to left
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('FONTNAME', (0,0), (-1,0), f"{MAIN_FONT}-Bold"),
        ('FONTSIZE', (0,0), (-1,0), 9),
        ('BOTTOMPADDING', (0,0), (-1,0), 6),
        ('BACKGROUND', (0,1), (-1,-1), colors.white),
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('BOX', (0,0), (-1,-1), 1, colors.black),
        ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor("#f7fafc")]), # Simpler row backgrounds
    ]))
    elements.append(financial_table)
    elements.append(Spacer(1, 6))

    # Additional explanations for financial summary
    elements.append(Paragraph("Açıklamalar:", normal_style))
    elements.append(Paragraph(
        "<b>Alt Toplam:</b> Projenin tüm malzeme ve işçilik maliyetlerinin KDV ve kar marjı hariç toplamıdır.", normal_style
    ))
    elements.append(Paragraph(
        f"<b>Atık Maliyeti (%{FIRE_RATE*100:.0f}):</b> Malzeme kesimi, fire ve olası hatalar için eklenen maliyettir.", normal_style
    ))
    elements.append(Paragraph(
        "<b>Toplam Maliyet (Atık Dahil):</b> Atık maliyetinin eklenmesiyle elde edilen nihai maliyet miktarıdır.", normal_style
    ))
    elements.append(Paragraph(
        f"<b>Kar Oranı (%{profit_rate_input.value*100:.0f}):</b> Şirketimizin projeden elde etmeyi hedeflediği kar marjıdır.", normal_style
    ))
    elements.append(Paragraph(
        f"<b>KDV (%{vat_rate_input.value*100:.0f}):</b> Geçerli katma değer vergisi oranıdır.", normal_style
    ))
    elements.append(Paragraph(
        "<b>Satış Fiyatı (KDV Dahil) - Solar Hariç:</b> Solar enerji sistemi maliyeti hariç, müşteriye sunulan nihai satış fiyatıdır.", normal_style
    ))
    if project_details['solar']:
        elements.append(Paragraph(
            "<b>Solar Enerji Sistemi:</b> Müşteri tarafından talep edilen ek solar enerji sistemi maliyetidir.", normal_style
        ))
        elements.append(Paragraph(
            "<b>GENEL TOPLAM (Solar & KDV Dahil):</b> Solar enerji sistemi dahil olmak üzere projenin toplam satış fiyatıdır.", normal_style
        ))

    elements.append(Spacer(1, 24))


    # Build PDF
    doc.build(elements)

    buffer.seek(0)
    return buffer.getvalue()


# === UI COMPONENTS ===
# Customer Section
customer_section = widgets.VBox([
    customer_title,
    customer_note,
    widgets.HBox([customer_name, customer_company]),
    widgets.HBox([customer_address, customer_city]),
    widgets.HBox([customer_phone, customer_email])
], layout=widgets.Layout(margin='0 0 20px 0'))

# Project Details Section
project_details_section = widgets.VBox([
    widgets.HTML("<div class='section-title'>PROJECT DETAILS</div>"),
    structure_type,
    plasterboard_option,
    widgets.HBox([width_input, length_input, height_input]),
    room_config_input
], layout=widgets.Layout(margin='0 0 20px 0'))

# Windows and Doors Section
windows_doors_section = widgets.VBox([
    widgets.HTML("<div class='card'>"),
    widgets.HTML("<div class='section-title'>WINDOWS AND DOORS</div>"),
    widgets.HBox([window_input, window_size]),
    widgets.HBox([sliding_door_input, sliding_door_size]),
    widgets.HBox([wc_window_input, wc_window_size]),
    widgets.HBox([wc_sliding_door_input, wc_sliding_door_size]),
    widgets.HBox([door_input, door_size]),
    widgets.HTML("</div>")
], layout=widgets.Layout(margin='0 0 20px 0'))

# Additional Equipment Section
equipment_section = widgets.VBox([
    widgets.HTML("<div class='card'>"),
    widgets.HTML("<div class='section-title'>ADDITIONAL EQUIPMENT</div>"),
    widgets.HBox([kitchen_input, shower_input]),
    widgets.HBox([wc_ceramic_input, wc_ceramic_area]),
    widgets.HBox([electrical_installation_input, plumbing_installation_input]),
    transportation_input,
    heating_option,
    widgets.HBox([solar_option, solar_capacity, solar_price_display]),
    widgets.HTML("</div>")
], layout=widgets.Layout(margin='0 0 20px 0'))

# Financial Settings Section
financial_section = widgets.VBox([
    widgets.HTML("<div class='card'>"),
    widgets.HTML("<div class='section-title'>FINANCIAL SETTINGS</div>"),
    profit_rate_input,
    vat_rate_input,
    widgets.HTML("</div>")
], layout=widgets.Layout(margin='0 0 20px 0'))

# Customer Notes Section (Initially hidden, revealed if 'Custom' room config selected)
notes_section = widgets.VBox([
    widgets.HTML("<div class='card'>"),
    widgets.HTML("<div class='section-title'>CUSTOMER SPECIAL REQUESTS AND NOTES</div>"),
    customer_notes,
    widgets.HTML("</div>")
], layout=widgets.Layout(margin='0 0 20px 0'))

# Function to toggle visibility of notes section based on room_config_input
def toggle_notes_visibility(change):
    if change['new'] == 'Custom':
        notes_section.layout.display = 'flex' # Show the section
    else:
        notes_section.layout.display = 'none' # Hide the section

# Observe changes in room_config_input
room_config_input.observe(toggle_notes_visibility, names='value')

# Initial call to set visibility based on default value
toggle_notes_visibility({'new': room_config_input.value})


# Main UI
ui = widgets.VBox([
    dark_mode_button,
    customer_section,
    project_details_section,
    steel_profile_section, # RE-ADDED to UI
    windows_doors_section,
    equipment_section,
    financial_section,
    notes_section # Include notes section in UI
])

# Output area for results and PDF download
output_area = widgets.Output()

def on_calculate_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        try:
            results = calculate()
            cost_df = results['cost_breakdown']
            financial_df = results['financial_summary']
            profile_analysis_df = results['profile_analysis'] # profile_analysis_df re-added
            customer_info = results['customer_info']
            project_details = results['project_details']
            notes = results['notes']
            sales_price_excluding_solar = results['sales_price_excluding_solar']
            total_sales_price_with_solar = results['total_sales_price_with_solar']

            # Add total_sales_price_with_solar to project_details for customer PDF
            project_details['total_sales_price_with_solar'] = total_sales_price_with_solar

            display(HTML("<h3>Maliyet Dağılımı (Dahili Rapor)</h3>"))
            display(cost_df.style.set_table_attributes("class='dataframe'"))

            # Display profile analysis if it's not empty (i.e., Light Steel was selected)
            if not profile_analysis_df.empty:
                display(HTML("<h3>Çelik Profil Analizi (Dahili Rapor)</h3>"))
                display(profile_analysis_df.style.set_table_attributes("class='dataframe'"))

            display(HTML("<h3>Finansal Özet (Dahili Rapor)</h3>"))
            display(financial_df.style.set_table_attributes("class='price-table'"))

            # Generate PDFs
            logo_data = get_company_logo()

            # Format date for filename: DD-MM-YY
            date_str_filename = datetime.now().strftime('%d-%m-%y')
            # Sanitize customer name for filename
            sanitized_customer_name = customer_info['name'].replace(' ', '_').replace('/', '_').replace('\\', '_')

            # Internal Cost Report PDF
            company_name_sanitized = COMPANY_INFO["name"].replace(' ', '_').replace('/', '_').replace('\\', '_')
            internal_pdf_filename = f"{company_name_sanitized}_{sanitized_customer_name}_{date_str_filename}_Internal_Report.pdf"
            # profile_analysis_df re-added to arguments
            internal_pdf_bytes = create_internal_cost_report_pdf(cost_df, financial_df, profile_analysis_df, project_details, logo_data)
            display(HTML(f"<div class='pdf-container'><h4>Dahili Maliyet Raporu (Kayıtlarınız İçin)</h4>{create_pdf_download_link(internal_pdf_bytes, internal_pdf_filename)}</div>"))

            # Customer Proposal PDF
            customer_pdf_filename = f"{company_name_sanitized}_{sanitized_customer_name}_{date_str_filename}_Customer_Proposal.pdf"
            # Pass sales_price_excluding_solar for the main proposal price, and project_details now contains total_sales_price_with_solar
            customer_pdf_bytes = create_customer_proposal_pdf(sales_price_excluding_solar, project_details, notes, customer_info, logo_data)
            display(HTML(f"<div class='pdf-container'><h4>Müşteri Teklifi</h4>{create_pdf_download_link(customer_pdf_bytes, customer_pdf_filename)}</div>"))

        except Exception as e:
            display(HTML(f"<div class='warning'>Bir hata oluştu: {e}<br>An error occurred: {e}</div>"))
            # Optionally, print the full traceback for debugging in the console
            import traceback
            traceback.print_exc()

calculate_button = widgets.Button(
    description="Calculate & Generate Proposals",
    button_style='success',
    icon='calculator',
    layout=widgets.Layout(margin='20px 0 0 0', width='auto')
)
calculate_button.on_click(on_calculate_button_clicked)

display(ui, calculate_button, output_area)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Button(button_style='success', description='Calculate & Generate Proposals', icon='calculator', layout=Layout(…

Output()